# Fedbiomed Researcher base example with OPACUS

Use for developing (autoreloads changes made across packages)

In this notebook we show how `opacus` (https://opacus.ai/) can be used in Fed-BioMed. Opacus is a library which allows to train PyTorch models with differential privacy. We will train the basic MNIST example using two nodes.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-01-31 10:49:14,722 fedbiomed INFO - Component environment:
2022-01-31 10:49:14,723 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-31 10:49:28,897 fedbiomed INFO - Messaging researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x10393beb0>
2022-01-31 10:49:29,013 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-31 10:49:29,084 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / DEBUG - Message received: {'researcher_id': 'researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf', 'command': 'list'}
2022-01-31 10:49:29,094 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / DEBUG - Message received: {'researcher_id': 'researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf', 'command': 'list'}
2022-01-31 10:49:39,023 fedbiomed INFO - 
 Node: node_518f73bb-5268-4463-b280-4e2e11a9335f | Number of Datasets: 1 
+--------+-------------+---

{'node_518f73bb-5268-4463-b280-4e2e11a9335f': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}],
 'node_d9cb986f-6aa6-4268-9081-30ae1012369c': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28]}]}

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default)
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the function `make_private` from `opacus.privacy_engine`. Two hyperparameters should be defined:
* `noise_multiplier`: The ratio of the standard deviation of the Gaussian noise to the L2-sensitivity of the function to which the noise is added (How much noise to add)
* `max_grad_norm`: The maximum norm of the per-sample gradients. Any gradient with norm higher than this will be clipped to this value.

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
import torch.nn.functional as F
from fedbiomed.common.torchnn import TorchTrainingPlan
from fedbiomed.common.logger import logger
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from opacus import PrivacyEngine 
from typing import Union, List
from tqdm import tqdm

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
                "import torch.nn.functional as F",
                "from torch.utils.data import DataLoader",
                "from fedbiomed.common.logger import logger",
                "from typing import Union, List",
                "from tqdm import tqdm",
                "from opacus import PrivacyEngine",]
        self.add_dependency(deps)
        
        self.model = nn.Sequential(nn.Conv2d(1, 32, 3, 1),
                                  nn.ReLU(),
                                  nn.Conv2d(32, 64, 3, 1),
                                  nn.ReLU(),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.25),
                                  nn.Flatten(),
                                  nn.Linear(9216, 128),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(128, 10),
                                  nn.LogSoftmax(dim=1))
        
        self.noise_multiplier = model_args['noise_multiplier']
        self.max_grad_norm = model_args['max_grad_norm']

    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss
    
    def training_routine(self,
                         epochs: int = 2,
                         log_interval: int = 10,
                         lr: Union[int, float] = 1e-3,
                         batch_size: int = 48,
                         batch_maxnum: int = 0,
                         dry_run: bool = False,
                         monitor=None):
        
        #if self.optimizer is None:
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

        # use_cuda = torch.cuda.is_available()
        # device = torch.device("cuda" if use_cuda else "cpu")
        #self.device = "cpu"
        
        training_data = self.training_data(batch_size=batch_size)
        
        # enter PrivacyEngine
        privacy_engine = PrivacyEngine()
        self.model, self.optimizer, training_data = privacy_engine.make_private(module=self.model,
                                                                    optimizer=self.optimizer,
                                                                    data_loader=training_data,
                                                                    noise_multiplier=self.noise_multiplier,
                                                                    max_grad_norm=self.max_grad_norm,
                                                                    )

        for epoch in range(1, epochs + 1):
            # (below) sampling data (with `training_data` method defined on
            # researcher's notebook)
            for batch_idx, (data, target) in enumerate(tqdm(training_data)):
                #self.model.train()  # model training
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()
                # (below) calling method `training_step` defined on
                # researcher's notebook
                res = self.training_step(data, target)
                res.backward()
                self.optimizer.step()

                # do not take into account more than batch_maxnum
                # batches from the dataset
                if (batch_maxnum > 0) and (batch_idx >= batch_maxnum):
                    #print('Reached {} batches for this epoch, ignore remaining data'.format(batch_maxnum))
                    logger.debug('Reached {} batches for this epoch, ignore remaining data'.format(batch_maxnum))
                    break

                if batch_idx % log_interval == 0:
                    logger.info('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch,
                        batch_idx * len(data),
                        len(training_data.dataset),
                        100 * batch_idx / len(training_data),
                        res.item()))

                    # Send scalar values via general/feedback topic
                    if monitor is not None:
                        monitor.add_scalar('Loss', res.item(), batch_idx, epoch)

                    if dry_run:
                        return

    def save(self, filename, params: dict = None) -> None:
        if params is not None:

            # params keys are changed by the privacy engine (as _module.param_key): should be re-named
            params_keys = list(params.keys())
            for key in params_keys:
                if '_module' in key:
                    newkey = key.replace('_module.', '')
                    params[newkey] = params.pop(key)
                    
            return(torch.save(params, filename))
        else:
            return torch.save(self.state_dict(), filename)

Writing /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/tmp2w4fr65n/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {'noise_multiplier':1., 'max_grad_norm':1.0}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 3

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-01-31 10:50:13,684 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-01-31 10:50:13,761 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / DEBUG - Message received: {'researcher_id': 'researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-31 10:50:13,768 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / DEBUG - Message received: {'researcher_id': 'researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-31 10:50:23,692 fedbiomed INFO - Node selected for training -> node_d9cb986f-6aa6-4268-9081-30ae1012369c
2022-01-31 10:50:23,693 fedbiomed INFO - Node selected for training -> node_518f73bb-5268-4463-b280-4e2e11a9335f
2022-01-31 10:50:23,706 fedbiomed INFO - Checking data quality of federated datasets...
2022-01-31 10:50:28,826 fedbiomed DEBUG - torchnn saved model filename: /Users/balelli/o

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [7]:
exp.run()

2022-01-31 10:50:37,458 fedbiomed INFO - Sampled nodes in round 0 ['node_d9cb986f-6aa6-4268-9081-30ae1012369c', 'node_518f73bb-5268-4463-b280-4e2e11a9335f']
01/31/2022 10:50:37:INFO:Sampled nodes in round 0 ['node_d9cb986f-6aa6-4268-9081-30ae1012369c', 'node_518f73bb-5268-4463-b280-4e2e11a9335f']
2022-01-31 10:50:37,460 fedbiomed INFO - Send message to node node_d9cb986f-6aa6-4268-9081-30ae1012369c - {'researcher_id': 'researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf', 'job_id': '1331f088-570c-4518-abf2-3bc0d3624d3f', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/31/my_model_fa62fb65-53cd-430f-803a-eff38ee80862.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/31/aggregated_params_init_84562606-77cf-4777-bd99-8f6f498cfcc3.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'n

2022-01-31 10:50:37,703 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf', 'job_id': '1331f088-570c-4518-abf2-3bc0d3624d3f', 'params_url': 'http://localhost:8844/media/uploads/2022/01/31/aggregated_params_init_84562606-77cf-4777-bd99-8f6f498cfcc3.pt', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 250}, 'training_data': {'node_518f73bb-5268-4463-b280-4e2e11a9335f': ['dataset_0dc58a75-fa9c-4744-821e-16afd239ba6b']}, 'model_args': {'noise_multiplier': 1.0, 'max_grad_norm': 1.0}, 'model_url': 'http://localhost:8844/media/uploads/2022/01/31/my_model_fa62fb65-53cd-430f-803a-eff38ee80862.py', 'model_class': 'MyTrainingPlan', 'command': 'train'}
01/31/2022 10:50:37:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_8cb51ba0-4a98-465b-949e-ca4e920f56bf', 'job_id': '13

2022-01-31 10:52:40,693 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [4500/60000 (7%)]	Loss: 2.191051
01/31/2022 10:52:40:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [4500/60000 (7%)]	Loss: 2.191051
2022-01-31 10:52:42,318 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [4590/60000 (7%)]	Loss: 2.092470
01/31/2022 10:52:42:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [4590/60000 (7%)]	Loss: 2.092470
2022-01-31 10:52:49,222 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.049824
01/31/2022 10:52:49:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.049824
2022-01-31 10:52:50,406 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [4500/60000 (8%)]	Loss: 2.03715

2022-01-31 10:54:36,272 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [9200/60000 (18%)]	Loss: 1.506714
01/31/2022 10:54:36:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [9200/60000 (18%)]	Loss: 1.506714
2022-01-31 10:54:40,757 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [12000/60000 (19%)]	Loss: 1.672460
01/31/2022 10:54:40:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [12000/60000 (19%)]	Loss: 1.672460
2022-01-31 10:54:43,385 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [14160/60000 (19%)]	Loss: 1.497844
01/31/2022 10:54:43:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [14160/60000 (19%)]	Loss: 1.497844
2022-01-31 10:54:49,732 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / DEBUG - Reached 250 batches for this epoch,

2022-01-31 10:56:32,267 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 2 [5520/60000 (10%)]	Loss: 1.573025
01/31/2022 10:56:32:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 2 [5520/60000 (10%)]	Loss: 1.573025
2022-01-31 10:56:32,571 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 2 [5040/60000 (10%)]	Loss: 1.148585
01/31/2022 10:56:32:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 2 [5040/60000 (10%)]	Loss: 1.148585
2022-01-31 10:56:40,113 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 2 [8190/60000 (10%)]	Loss: 1.560230
01/31/2022 10:56:40:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 2 [8190/60000 (10%)]	Loss: 1.560230
2022-01-31 10:56:40,846 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 2 [5720/60000 (10%)]	Loss: 

...

01/31/2022 11:14:59:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [14080/60000 (18%)]	Loss: 1.304276
2022-01-31 11:14:59,527 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [8580/60000 (18%)]	Loss: 2.044783
01/31/2022 11:14:59:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [8580/60000 (18%)]	Loss: 2.044783
2022-01-31 11:15:06,332 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [10810/60000 (18%)]	Loss: 1.128414
01/31/2022 11:15:06:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 1 [10810/60000 (18%)]	Loss: 1.128414
2022-01-31 11:15:07,101 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [9430/60000 (18%)]	Loss: 1.531657
01/31/2022 11:15:07:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 1 [9430/60000 (18%)]	Loss: 1.531657
2022

01/31/2022 11:16:46:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 2 [6000/60000 (8%)]	Loss: 0.812242
2022-01-31 11:16:53,984 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 2 [5500/60000 (9%)]	Loss: 1.324986
01/31/2022 11:16:53:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 2 [5500/60000 (9%)]	Loss: 1.324986
2022-01-31 11:16:54,758 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 2 [5280/60000 (9%)]	Loss: 0.842654
01/31/2022 11:16:54:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 2 [5280/60000 (9%)]	Loss: 0.842654
2022-01-31 11:17:01,337 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 2 [6720/60000 (10%)]	Loss: 1.124550
01/31/2022 11:17:01:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 2 [6720/60000 (10%)]	Loss: 1.124550
2022-01-31 1

01/31/2022 11:18:38:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-31 11:18:39,581 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 1.251405
01/31/2022 11:18:39:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 1.251405
2022-01-31 11:18:40,633 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / DEBUG - Reached 250 batches for this epoch, ignore remaining data
01/31/2022 11:18:40:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-31 11:18:41,522 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 1.596982
01/31/2022 11:18:41:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 3 [0/60000 (0%)]	Loss: 1.

01/31/2022 11:20:38:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 3 [5720/60000 (10%)]	Loss: 1.439171
2022-01-31 11:20:46,075 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 3 [6440/60000 (11%)]	Loss: 1.319028
01/31/2022 11:20:46:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 3 [6440/60000 (11%)]	Loss: 1.319028
2022-01-31 11:20:47,198 fedbiomed INFO - log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 3 [6580/60000 (11%)]	Loss: 1.263639
01/31/2022 11:20:47:INFO:log from: node_d9cb986f-6aa6-4268-9081-30ae1012369c / INFO - Train Epoch: 3 [6580/60000 (11%)]	Loss: 1.263639
2022-01-31 11:20:53,721 fedbiomed INFO - log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 3 [7650/60000 (12%)]	Loss: 0.679676
01/31/2022 11:20:53:INFO:log from: node_518f73bb-5268-4463-b280-4e2e11a9335f / INFO - Train Epoch: 3 [7650/60000 (12%)]	Loss: 0.679676
2022-01

01/31/2022 11:22:25:INFO:Downloading model params after training on node_d9cb986f-6aa6-4268-9081-30ae1012369c - from http://localhost:8844/media/uploads/2022/01/31/node_params_0500313e-7510-4107-812c-01039d422d12.pt
2022-01-31 11:22:25,547 fedbiomed INFO - Nodes that successfully reply in round 2 ['node_518f73bb-5268-4463-b280-4e2e11a9335f', 'node_d9cb986f-6aa6-4268-9081-30ae1012369c']
01/31/2022 11:22:25:INFO:Nodes that successfully reply in round 2 ['node_518f73bb-5268-4463-b280-4e2e11a9335f', 'node_d9cb986f-6aa6-4268-9081-30ae1012369c']
2022-01-31 11:22:26,704 fedbiomed INFO - Saved aggregated params for round 2 in /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0001/aggregated_params_8952d115-b674-4856-84eb-fb46e23c093e.pt
01/31/2022 11:22:26:INFO:Saved aggregated params for round 2 in /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0001/aggregated_params_8952d115-b674-4856-84eb-fb46e23c093e.pt


Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [8]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe


List the training rounds :  dict_keys([0, 1, 2])

List the nodes for the last training round and their timings : 
	- node_518f73bb-5268-4463-b280-4e2e11a9335f :    
		rtime_training=622.83 seconds    
		ptime_training=696.41 seconds    
		rtime_total=635.52 seconds
	- node_d9cb986f-6aa6-4268-9081-30ae1012369c :    
		rtime_training=625.07 seconds    
		ptime_training=700.91 seconds    
		rtime_total=635.93 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_0dc58a75-fa9c-4744-821e-16afd239ba6b,node_518f73bb-5268-4463-b280-4e2e11a9335f,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.1698,...","{'rtime_training': 622.826828749, 'ptime_train..."
1,True,,dataset_2adcd6bd-1086-41ed-8c8f-93091cc97f49,node_d9cb986f-6aa6-4268-9081-30ae1012369c,/Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed...,"{'model._module.0.weight': [[tensor([[-0.1693,...","{'rtime_training': 625.0724522450002, 'ptime_t..."


Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [9]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1, 2])

Access the federated params for the last training round :
	- params_path:  /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/experiments/Experiment_0001/aggregated_params_8952d115-b674-4856-84eb-fb46e23c093e.pt
	- parameter data:  odict_keys(['model.0.weight', 'model.0.bias', 'model.2.weight', 'model.2.bias', 'model.7.weight', 'model.7.bias', 'model.10.weight', 'model.10.bias'])


# Testing

We define a little testing routine to extract the accuracy metrics on the testing dataset

In [10]:
import torch
import torch.nn.functional as F


def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'

    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/len(data_loader.dataset)

    return(test_loss, accuracy)

In [11]:
from torchvision import datasets, transforms
import os

local_mnist = os.path.join(environ['TMP_DIR'], 'local_mnist')

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

test_set = datasets.MNIST(root = local_mnist, download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=True)

fed_model = exp.model_instance
fed_model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

acc_federated = testing_Accuracy(fed_model, test_loader)

print('\nAccuracy federated training:  {:.4f}'.format(acc_federated[1]))

print('\nError federated training:  {:.4f}'.format(acc_federated[0]))

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-images-idx3-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/balelli/ownCloud/INRIA_EPIONE/FedBioMed/fedbiomed/var/tmp/local_mnist/MNIST/raw

Processing...


/Users/balelli/miniconda3/envs/fedbiomed-researcher/lib/python3.9/site-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!

Accuracy federated training:  86.7000

Error federated training:  0.5476
